[View in Colaboratory](https://colab.research.google.com/github/ebgv/Planet--Understanding-the-Amazon-from-Space/blob/master/model2_hidden_layers.ipynb)

In [1]:
# check the environment 

%ls

data/  sample_data/


In [2]:
# check if the data is in the environment 

%cd /content/data/sample/test/cloudy
%ls

/content/data/sample/test/cloudy
train_10043.jpg  train_17597.jpg  train_27945.jpg  train_37889.jpg
train_10128.jpg  train_1776.jpg   train_28182.jpg  train_37929.jpg
train_10177.jpg  train_1786.jpg   train_28244.jpg  train_37942.jpg
train_10225.jpg  train_18134.jpg  train_28318.jpg  train_37980.jpg
train_10246.jpg  train_18253.jpg  train_28454.jpg  train_38089.jpg
train_10265.jpg  train_18344.jpg  train_28463.jpg  train_3815.jpg
train_10293.jpg  train_18513.jpg  train_28488.jpg  train_38278.jpg
train_10389.jpg  train_18551.jpg  train_28657.jpg  train_38329.jpg
train_10485.jpg  train_18572.jpg  train_28765.jpg  train_38400.jpg
train_10547.jpg  train_18913.jpg  train_28974.jpg  train_38405.jpg
train_10552.jpg  train_18942.jpg  train_29002.jpg  train_38657.jpg
train_10563.jpg  train_19069.jpg  train_29014.jpg  train_38807.jpg
train_10598.jpg  train_19363.jpg  train_2904.jpg   train_38808.jpg
train_10895.jpg  train_1938.jpg   train_29182.jpg  train_38823.jpg
train_10908.jpg  train_19504.j

In [3]:
# check if the data is in the environment 

%cd /content/data/sample/train/cloudy
%ls

/content/data/sample/train/cloudy
train_10021.jpg  train_18954.jpg  train_27884.jpg  train_37070.jpg
train_10070.jpg  train_18976.jpg  train_2791.jpg   train_37171.jpg
train_10081.jpg  train_19000.jpg  train_2796.jpg   train_37175.jpg
train_10096.jpg  train_19027.jpg  train_28061.jpg  train_37206.jpg
train_10103.jpg  train_19085.jpg  train_28072.jpg  train_37260.jpg
train_10166.jpg  train_19110.jpg  train_28089.jpg  train_37294.jpg
train_10173.jpg  train_19117.jpg  train_28101.jpg  train_37305.jpg
train_101.jpg    train_19130.jpg  train_28197.jpg  train_37350.jpg
train_10201.jpg  train_1916.jpg   train_28242.jpg  train_37381.jpg
train_10224.jpg  train_1921.jpg   train_28272.jpg  train_37423.jpg
train_10227.jpg  train_19285.jpg  train_28284.jpg  train_37445.jpg
train_10231.jpg  train_19288.jpg  train_28292.jpg  train_37471.jpg
train_10263.jpg  train_19299.jpg  train_28309.jpg  train_37494.jpg
train_10269.jpg  train_1933.jpg   train_28329.jpg  train_37495.jpg
train_10326.jpg  train_19347

# Imports 

In [4]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x56af6000 @  0x7fb2a34d82a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [5]:
!pip install -U bcolz

    100% |████████████████████████████████| 1.5MB 13.6MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [6]:
!pip install Pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 1.2MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time

# GPU Settings 

In [8]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

dtype = torch.FloatTensor
if use_gpu:
    dtype = torch.cuda.FloatTensor

Using gpu: True 


# Data Loader

In [0]:
# loader taken from class example: parameters to verify

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # statistics from imagenet ? 

prep1 = transforms.Compose([
                transforms.CenterCrop(224), # default cropping 
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
data_dir = '/content/data/sample'
#data_dir = '/content/data'

#batch_size = 4
batch_size = 64

In [0]:
# dictionary of datasets - in this case only train data set 
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'test']}

In [12]:
# train labels 
print(dsets['train'].imgs[:5]) # 5 first images and labels 
print(dsets['train'].imgs[len(dsets['train'])-5:]) # 5 last images and labels 

[('/content/data/sample/train/clear/train_10.jpg', 0), ('/content/data/sample/train/clear/train_10001.jpg', 0), ('/content/data/sample/train/clear/train_10013.jpg', 0), ('/content/data/sample/train/clear/train_10014.jpg', 0), ('/content/data/sample/train/clear/train_10038.jpg', 0)]
[('/content/data/sample/train/partly_cloudy/train_9849.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9851.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9892.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9921.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9997.jpg', 3)]


In [13]:
# test labels 
print(dsets['test'].imgs[:5]) # 5 first images and labels 
print(dsets['test'].imgs[len(dsets['test'])-5:]) # 5 last images and labels 

[('/content/data/sample/test/clear/train_10077.jpg', 0), ('/content/data/sample/test/clear/train_10155.jpg', 0), ('/content/data/sample/test/clear/train_10198.jpg', 0), ('/content/data/sample/test/clear/train_10315.jpg', 0), ('/content/data/sample/test/clear/train_10369.jpg', 0)]
[('/content/data/sample/test/partly_cloudy/train_9552.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_969.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9720.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9772.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_994.jpg', 3)]


In [14]:
dset_classes = dsets['train'].classes
dset_classes # binary classes to begin 

['clear', 'cloudy', 'haze', 'partly_cloudy']

In [0]:
# dictionary of data loaders - again only train for now 
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=True, num_workers=0)
                for x in ['train', 'test']}

In [16]:
print(len(dsets['train']))
print(len(dsets['test']))

6400
1600


# Model: A bit more complex one

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(4096, 256)
        self.fc2 = nn.Linear(256, 4)

    def forward(self, x):
        # First Conv block:
        x = self.conv1(x)
        x = F.max_pool2d(x, 5)
        x = F.relu(x)
        
        # Second Conv block:
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = F.max_pool2d(x, 5)
        x = F.relu(x)
        
        # Flatten:
        x = x.view(x.size(0), -1)
        
        # Classifier:
        ## First layer:
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # Second layer:
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)

        return x

# Training Framework

In [0]:
def train(model,data_loader,loss_fn,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()    
                
            # batch_size ?
            bs = labels.size(0)
            
            # define the loss to minimize
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            # define the optimizer
            optimizer = optimizer
            optimizer.zero_grad()
            # backpropagation
            loss.backward()
            optimizer.step()
            
            # predictions to get statistics 
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == labels.data)
            size += bs
        # epoch statistics     
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
    return loss_train, acc_train

# Running training epochs

In [37]:
# instanciate the model 
model = NNet()
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()

# choose the appropriate loss
loss_fn = torch.nn.CrossEntropyLoss()
# learning rate 
learning_rate = 1e-3
# your SGD optimizer
optimizer_cl = torch.optim.Adam(model.parameters(), lr=learning_rate)


# and train for 10 epochs
l_t, a_t = train(model, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 10)

Train - Loss: 0.0146 Acc: 0.6061
Train - Loss: 0.0134 Acc: 0.6550
Train - Loss: 0.0123 Acc: 0.6891
Train - Loss: 0.0119 Acc: 0.6966
Train - Loss: 0.0117 Acc: 0.7067
Train - Loss: 0.0110 Acc: 0.7344
Train - Loss: 0.0105 Acc: 0.7484
Train - Loss: 0.0104 Acc: 0.7486
Train - Loss: 0.0101 Acc: 0.7612
Train - Loss: 0.0095 Acc: 0.7720


# Testing 

In [0]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0

    for data in data_loader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
            
        bs = labels.size(0)
                
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        _,preds = torch.max(outputs.data,1)
        
        # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == labels.data)
        size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [39]:
# using 'hidden'

test(model, dset_loaders['test'])

Test - Loss: 0.0099 Acc: 0.7631
